In [1]:
import random
import numpy as np

from RobotEnviroment.init import *
from RobotEnviroment.arenas import RectangularArena
from PointClouds.InteractionPointFinder import getPushPoints
from PointClouds.Registration import *
from PathPlanning.scanning import *
from PathPlanning.motion import *
from utils.debug import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Initialise Robot and Configuration

In [2]:
ON_REAL = False
ROBOT_VELOCITY = 1. # m/s ???
INITIAL_OBJ_POS = np.array([-.50, -.1, .69])

allowedViewAngleSegments = [[0., np.pi*.25], [.75*np.pi, 1.25*np.pi], [1.75*np.pi, np.pi*2]]

C = setup_config(INITIAL_OBJ_POS, ON_REAL)
bot = startup_robot(C, ON_REAL)
visualizeViewAreas(C, INITIAL_OBJ_POS, allowedViewAngleSegments)

Define an arena to stop the robot from moving the object too far

In [3]:
TABLE_CENTER = np.array([-.23, -.16, .651])
TABLE_DIMS = np.array([.89, .55])
ROBOT_POS = np.array([-.03, -.24, .651])
ROBOT_RING = .29

arena = RectangularArena(middleP=TABLE_CENTER, width=TABLE_DIMS[0], height=TABLE_DIMS[1], middlePCirc=ROBOT_POS, innerR=ROBOT_RING)
arena.plotArena(C)

### Main pushing loop
- Look towards the predicted object position
- Get the object's point cloud and calculate its normals
- Choose a random pushing point in the point cloud
- Calculate the pushing motion
- Push if the motion is feasible

In [4]:
NUMBER_OF_PUSH_TRIALS = 10
predObjPos = INITIAL_OBJ_POS

pointClouds = [] # Stores the world position of the point cloud and the point cloud {"world_position": [x, y, z], "pc": np.array([])}
minNumScans = 2 # Minimum nuber of point cloud scans until we start merging them
fullPC = np.array([])
maxForces = []

for i in range(NUMBER_OF_PUSH_TRIALS):

    print("Starting Trial Number ", i+1)

    # Scan object at it's predicted position and get it's possible push points
    print("Scanning object...")
    lookAngle = giveRandomAllowedAngle(allowedViewAngleSegments)
    lookAtObjFromAngle(predObjPos, bot, C, lookAngle, velocity=ROBOT_VELOCITY, verbose=2)
    predObjPos, pointCloud = getScannedObject(bot, C, arena)
    if not len(predObjPos):
        print ("Lost the Object!")
        break
    push_points  = getPushPoints(pointCloud, verbose=0)

    # Store the object's point cloud
    print("Storing point cloud")
    for j, _ in enumerate(pointCloud):
        pointCloud[j] -= predObjPos
    pointClouds.append(pointCloud)

    if len(pointClouds) >= minNumScans:
        fullPC = joinOffsetPCS(pointClouds.copy(), verbose=0)

    # Try to push object
    for _ in range(5): # This counts the number of attempts to push an object from the current view angle. If it reaches 5 we try a new angle
        if not len(push_points):
            print("No viable push points found!")
            break

        pushP = random.choice(push_points)
        # push_points.remove(pushP)

        print("Trying to push obj...")
        start, end, direction = arena.generate_waypoints(pushP[0], C)
        success, maxForce = specialPush(bot, C, direction, verbose=2)
        if success:
            print("Success! :)")
            predObjPos = end # This shuld be offset by the width of the object!
            maxForces.append(maxForce)
            break
        print("Failed! :(")

Starting Trial Number  1
Scanning object...
{ time: 0.030997, evals: 24, done: 1, feasible: 1, sos: 1512.89, f: 0, ineq: 0.000176046, eq: 0.00136682 }
1 0.751032
-- WARNING:kin.cpp:getFrame:393(-1) cannot find frame named 'pcl'
Storing point cloud
Trying to push obj...
-- WARNING:kin.cpp:getFrame:393(-1) cannot find frame named 'startWayPoint'
-- WARNING:kin.cpp:getFrame:393(-1) cannot find frame named 'pushWayPoint'
-- WARNING:kin.cpp:getFrame:393(-1) cannot find frame named 'endWayPoint'
Error while executing robot movement!
Failed! :(
Trying to push obj...
Error while executing robot movement!
Failed! :(
Trying to push obj...
Error while executing robot movement!
Failed! :(
Trying to push obj...
Error while executing robot movement!
Failed! :(
Trying to push obj...
Error while executing robot movement!
Failed! :(
Starting Trial Number  2
Scanning object...
{ time: 0.055167, evals: 14, done: 1, feasible: 1, sos: 176.465, f: 0, ineq: 0, eq: 0.000394002 }
0.66427 1
Storing point cloud


In [5]:
bot.home(C)

In [6]:
del bot
del C

-- bot.cpp:~BotOp:118(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:149(0) shutting down Simulation
